In [1]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4e914tyh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4e914tyh
  Resolved https://github.com/huggingface/transformers to commit 3f69f415adcbdaedec154ba8eac220ef3276975d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8404406 sha256=a22d0eb0e7b3236214781af5406057ff0b44f229f8d33bcf8264369d9215a52c
  Stored in directory: /tmp/pip-ephem-wheel-cache-llf5rs4u/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Succ

In [2]:
# loading datasets on which we want to train tokenizer
from datasets import load_dataset
raw_dataset  = load_dataset("code_search_net", "python")
raw_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [7]:
raw_dataset["train"][0]["whole_func_string"]                     # inspecting training columns of dataset

"def update(self, field_dict, where_clause=None):\n        '''\n        update db entry\n\n        :param field_dict: dictionary of fields and values\n        :param where_clause: where clause for the update\n        '''\n        query = '''\n        UPDATE %s SET %s\n        ''' % (\n            self._name,\n            ','.join('%s=:%s' % (k, k) for k in field_dict)\n        )\n        if where_clause:\n            query += ' WHERE %s' % (where_clause)\n        self._cursor.execute(query, field_dict)\n        self._connection.commit()"

In [8]:
def get_training_corpus(raw_datasets):
  for start_idx in range(0,len(raw_datasets["train"]),1000):                          # taking data with 1k jump steps
    data = raw_datasets["train"][start_idx: start_idx+1000]["whole_func_string"]      # taking data between jump steps
    yield data                                                                        # yield return data in iterable way

In [11]:
training_corpus = get_training_corpus(raw_dataset)                                    # making data ready to train tokenizer

In [16]:
next(iter(training_corpus))                                                           # inspecting corpus

['def fbeta_score(df, col_true=None, col_pred=\'precision_result\', beta=1.0, pos_label=1, average=None):\n    r"""\n    Compute f-beta score of a predicted DataFrame. f-beta is defined as\n\n    .. math::\n\n        \\frac{1 + \\beta^2 \\cdot precision \\cdot recall}{\\beta^2 \\cdot precision + recall}\n\n    F-beta score is a generalization of f-1 score.\n\n    :Parameters:\n        - **df** - predicted data frame\n        - **col_true** - column name of true label\n        - **col_pred** - column name of predicted label, \'prediction_result\' by default.\n        - **pos_label** - denote the desired class label when ``average`` == `binary`\n        - **average** - denote the method to compute average.\n    :Returns:\n        Recall score\n    :Return type:\n        float | numpy.array[float]\n\n    The parameter ``average`` controls the behavior of the function.\n\n    * When ``average`` == None (by default), f-beta of every class is given as a list.\n\n    * When ``average`` == \'b

In [17]:
# training a new tokenizer from pretrained tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#observing tokenizer pretrained on not so specific data
tokenizer.tokenize(raw_dataset["train"][0]["whole_func_string"] )

In [19]:
# training new tokenizer from pretrained tokenizer on our specific data
new_tokenizer = tokenizer.train_new_from_iterator(training_corpus, 52000)

In [44]:
tokenizer.tokenize(raw_dataset["train"][5]["whole_func_string"])                               # inspecting result

['def',
 'Ġ_',
 'serial',
 'ize',
 '_',
 'dict',
 '(',
 'cl',
 's',
 ',',
 'Ġdata',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 "Ġ'",
 "''",
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġserial',
 'izes',
 'Ġa',
 'Ġdictionary',
 'ĊĊ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ:',
 'param',
 'Ġdata',
 ':',
 'Ġdata',
 'Ġto',
 'Ġserial',
 'ize',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 "Ġ'",
 "''",
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġb',
 '64',
 'en',
 'code',
 '(',
 'z',
 'lib',
 '.',
 'comp',
 'ress',
 '(',
 'c',
 'Pick',
 'le',
 '.',
 'd',
 'umps',
 '(',
 'data',
 ',',
 'Ġprotocol',
 '=',
 '2',
 '))',
 ').',
 'dec',
 'ode',
 '()']

In [45]:
new_tokenizer.tokenize(raw_dataset["train"][5]["whole_func_string"])                                # inspecting result


['def',
 'Ġ_',
 'serialize',
 '_',
 'dict',
 '(',
 'cls',
 ',',
 'Ġdata',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 "Ġ'''",
 'ĊĠĠĠĠĠĠĠ',
 'Ġserializes',
 'Ġa',
 'Ġdictionary',
 'ĊĊĠĠĠĠĠĠĠ',
 'Ġ:',
 'param',
 'Ġdata',
 ':',
 'Ġdata',
 'Ġto',
 'Ġserialize',
 'ĊĠĠĠĠĠĠĠ',
 "Ġ'''",
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġb',
 '64',
 'encode',
 '(',
 'zlib',
 '.',
 'compress',
 '(',
 'c',
 'Pickle',
 '.',
 'dumps',
 '(',
 'data',
 ',',
 'Ġprotocol',
 '=',
 '2',
 '))).',
 'decode',
 '()']

In [48]:
from huggingface_hub import notebook_login

notebook_login()

In [49]:
new_tokenizer.push_to_hub("tokenizer_code_search_net_python")                                 # pushing tokenizer to hub for reusing

CommitInfo(commit_url='https://huggingface.co/Utshav/tokenizer_code_search_net_python/commit/85d3598bd668abe98998a8cadfe67b36406f187e', commit_message='Upload tokenizer', commit_description='', oid='85d3598bd668abe98998a8cadfe67b36406f187e', pr_url=None, pr_revision=None, pr_num=None)